In [1]:
import keras
import matplotlib.image as mpimg
import numpy
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import tensorflow as tf
import matplotlib.colors as colors

Using TensorFlow backend.
/Users/inesbahej/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 40
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16 # 64
NUM_EPOCHS = 50 #5
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 1000
IMG_PATCH_SIZE = 1

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - numpy.min(img)
    rimg = (rimg / numpy.max(rimg) * PIXEL_DEPTH).round().astype(numpy.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg


def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch


    
def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    #print(gt_imgs[0].shape)
    gt_exp = numpy.expand_dims(gt_imgs, axis=3)
    #print(tmp.shape)
    IMG_WIDTH = gt_exp[0].shape[0]
    IMG_HEIGHT = gt_exp[0].shape[1]
    print(IMG_WIDTH, IMG_HEIGHT)
    gt_patches = [img_crop(gt_exp[i], IMG_WIDTH, IMG_HEIGHT) for i in range(num_images)]
    data = [gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))]
    #labels = numpy.asarray([value_to_class(numpy.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    #return labels.astype(numpy.float32)
    return numpy.asarray(data)


def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    print(imgs[0].shape)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    #N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [img_crop(imgs[i], IMG_WIDTH, IMG_HEIGHT) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return numpy.asarray(data)
    #return imgs
    
    

def value_to_class(v):
    foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
    df = numpy.sum(v)
    if df > foreground_threshold:
        return [0, 1]
    else:
        return [1, 0]


In [3]:
from sklearn.model_selection import train_test_split

root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

train_data = extract_data(image_dir, TRAINING_SIZE)

print(train_data.shape)
train_labels = extract_labels(gt_dir, TRAINING_SIZE)
print(train_labels.shape)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Loading training/images/satImage_001.png
Loading training/images/satImage_002.png
Loading training/images/satImage_003.png
Loading training/images/satImage_004.png
Loading training/images/satImage_005.png
Loading training/images/satImage_006.png
Loading training/images/satImage_007.png
Loading training/images/satImage_008.png
Loading training/images/satImage_009.png
Loading training/images/satImage_010.png
Loading training/images/satImage_011.png
Loading training/images/satImage_012.png
Loading training/images/satImage_013.png
Loading training/images/satImage_014.png
Loading training/images/satImage_015.png
Loading training/images/satImage_016.png
Loading training/images/satImage_017.png
Loading training/images/satImage_018.png
Loading training/images/satImage_019.png
Loading training/images/satImage_020.png
Loading training/images/satImage_021.png
Loading training/images/satImage_022.png
Loading training/images/satImage_023.png
Loading training/images/satImage_024.png
Loading training

In [3]:
import os 
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
from keras.models import *
from keras.layers import Input, merge, Conv2D, Conv2DTranspose,MaxPooling2D,Activation, UpSampling2D, Dropout, Cropping2D, concatenate, Dense, Flatten
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.constraints import maxnorm
from keras.utils import to_categorical

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data




class Cnn:
    
    def __init__(self):   
        self.window_size = 64
        self.patch_size = 16
        self.pad_val = int((self.window_size-self.patch_size)/2)  
        self.build_model()
        
    def build_model(self):
        window_size = self.window_size
        patch_size = self.patch_size
        
        self.model = Sequential()
        self.model.add(Conv2D(32, (3, 3), input_shape=(window_size, window_size, 3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))


        self.model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Flatten())
        self.model.add(Dropout(0.5))
        self.model.add(Dense(2, activation='sigmoid'))
        
        
    def train(self, data, labels):
   
        window_size = self.window_size
        patch_size = self.patch_size
        batch_size = 128
        num_classes = 2

        
        samples_per_epoch = data.shape[0]*data.shape[1]*data.shape[2]//256

        # To use the window sliding method we need to use padding to work on the corners
        pad_val = int((window_size-patch_size)/2)  
        
        X = np.zeros( (data.shape[0],data.shape[1] + 2*pad_val, data.shape[2] + 2*pad_val, data.shape[3]) )
        Y = np.zeros( (labels.shape[0], labels.shape[1] + 2*pad_val, labels.shape[2] +2*pad_val) )
        
        for i in range(data.shape[0]):
            #X[i] = np.lib.pad(data[i], ((pad_val, pad_val), (pad_val, pad_val), (0, 0)), 'symmetric')
            #Y[i] = np.lib.pad(labels[i], ((pad_val, pad_val), (pad_val, pad_val)), 'symmetric')
            X[i] = np.lib.pad(data[i], ((pad_val, pad_val), (pad_val, pad_val), (0, 0)), 'reflect')
            Y[i] = np.lib.pad(labels[i], ((pad_val, pad_val), (pad_val, pad_val)), 'reflect')
        data = X
        labels = Y
            

        self.model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=0.001),
                      metrics=['accuracy'])

        np.random.seed(1)
        
        def value_to_class(v):
            foreground_threshold = 0.25
            df = np.array(v)
            if df > foreground_threshold:
                return 1
            else:
                return 0
            
        def gen():
            
            while True:
                batch_features = numpy.zeros((batch_size, window_size, window_size, 3))
                batch_labels = numpy.zeros((batch_size,2))
                for i in range(batch_size):
                    idx = numpy.random.choice(data.shape[0])
                    
                    wnd_center = numpy.random.randint(int(window_size/2), data[idx].shape[0]-int(window_size/2),2)
                    
                    batch_features[i] = data[idx][wnd_center[0] - int(window_size/2): wnd_center[0] + int(window_size/2),
                                                  wnd_center[1] - int(window_size/2): wnd_center[1] + int(window_size/2)]
                    
                    gt_patch = labels[idx][wnd_center[0] - int(patch_size/2): wnd_center[0] + int(window_size/2),
                                              wnd_center[1] - int(patch_size/2): wnd_center[1] + int(window_size/2)]
                    
                    label = value_to_class(numpy.mean(gt_patch))
                    batch_labels[i] =  to_categorical(label, 2) 
                    
                    
                yield (batch_features, batch_labels)

      

        #self.model.fit_generator(generator(),validation_data=generator(data,labels),validation_steps=0.1,
                        #samples_per_epoch=1000,
                        #   nb_epoch=20,
                        #   verbose=1)

        self.model.fit_generator(gen(),
                            steps_per_epoch=samples_per_epoch//batch_size,
                            nb_epoch=20,
                            verbose=1)


In [4]:
model = Cnn()
model.build_model()
print(model.model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
__________

In [5]:
import numpy as np


# Load the training set
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = 20 #len(files)
print("Loading " + str(n) + " images")
imgs = np.asarray([load_image(image_dir + files[i]) for i in range(n)])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = np.asarray([load_image(gt_dir + files[i]) for i in range(n)])


Loading 20 images
Loading 20 images


In [6]:
model.train(imgs, gt_imgs)

/Users/inesbahej/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:122: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/inesbahej/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:122: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=97, verbose=1, epochs=20)`


Epoch 1/20
97/97 [==============================] - 131s 1s/step - loss: 0.5861 - acc: 0.6876
Epoch 2/20
97/97 [==============================] - 144s 1s/step - loss: 0.5117 - acc: 0.7261
Epoch 3/20
97/97 [==============================] - 126s 1s/step - loss: 0.4602 - acc: 0.7717
Epoch 4/20
97/97 [==============================] - 129s 1s/step - loss: 0.4204 - acc: 0.7970
Epoch 5/20
97/97 [==============================] - 128s 1s/step - loss: 0.3794 - acc: 0.8270
Epoch 6/20
97/97 [==============================] - 131s 1s/step - loss: 0.3356 - acc: 0.8484
Epoch 7/20
97/97 [==============================] - 126s 1s/step - loss: 0.3175 - acc: 0.8610
Epoch 8/20
97/97 [==============================] - 127s 1s/step - loss: 0.2789 - acc: 0.8783
Epoch 9/20
97/97 [==============================] - 126s 1s/step - loss: 0.2693 - acc: 0.8842
Epoch 10/20
97/97 [==============================] - 127s 1s/step - loss: 0.2481 - acc: 0.8972
Epoch 11/20
97/97 [==============================] - 127s 1

In [9]:
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n =  50 #len(files)
print("Loading " + str(n) + " images")
X_test = np.asarray([load_image(image_dir + files[i]) for i in range(50,len(files))])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
y_test = np.asarray([load_image(gt_dir + files[i]) for i in range(50,len(files))])

scores = model.model.evaluate(X_test, y_test, verbose=1)

Loading 50 images
Loading 50 images


ValueError: Error when checking input: expected conv2d_4_input to have shape (None, 64, 64, 3) but got array with shape (50, 400, 400, 3)